In [49]:
import os
import time
import sys
import json
import requests
import datetime
from collections import OrderedDict
import pandas as pd


    
key = os.environ.get('YT_API')

from runtimestamp.runtimestamp import runtimestamp
runtimestamp()

Updated 2018-06-02 14:33:57.440545
By mab1242
Using Python 3.6.3
On Linux-3.10.0-514.10.2.el7.x86_64-x86_64-with-centos-7.3.1611-Core


In [50]:
def parse_comment_metadata(item):
    """
    Parses JSON object for comment metadata

    """
    
    if item['snippet'].get('topLevelComment'):
        item = item['snippet']['topLevelComment']
        
    comment_meta = OrderedDict(
        
        commenter_channel_url = item["snippet"].get("authorChannelUrl"),
        commenter_channel_display_name = item['snippet'].get('authorDisplayName'),
        comment_id = item.get("id"),
        comment_like_count = item["snippet"].get("likeCount"),
        comment_publish_date = parse_yt_datetime(item["snippet"].get("publishedAt")),
        text = item["snippet"].get("textDisplay"),
        video_id = item["snippet"].get("videoId"),
        commenter_rating = item["snippet"].get("viewerRating"),
        comment_parent_id = item["snippet"].get("parentId"),
        collection_date = datetime.datetime.now()
        
    )
    
    return comment_meta

In [51]:
def parse_yt_datetime(date_str):
    date = None
    if date_str:
        try: 
            date = datetime.datetime.strptime(date_str,"%Y-%m-%dT%H:%M:%S.%fZ")
        except:
            pass
    return date

In [69]:
def get_comments(video_id, key, get_replies=True):
    
    comments = []
    
    init_http_endpoint = ("https://www.googleapis.com/youtube/v3/commentThreads?"
                         "part=snippet&textFormat=plainText&maxResults=100&"
                         "videoId={}&key={}".format(video_id,key))
    
    response = requests.get(init_http_endpoint)
    
    json_doc = json.loads(response.text)
    
    comments = [parse_comment_metadata(comment) for comment in json_doc['items']]
    
    while json_doc.get('nextPageToken'):
        token = json_doc.get('nextPageToken')
        new_http_endpoint = ("https://www.googleapis.com/youtube/v3/commentThreads?"
                         "part=snippet&textFormat=plainText&maxResults=100&"
                         "videoId={}&key={}&pageToken={}".format(video_id,key,token))
        response = requests.get(new_http_endpoint)
        json_doc = json.loads(response.text)
        comments.extend([parse_comment_metadata(comment) for comment in json_doc['items']])
        
    if get_replies:
        for comment in comments:
            reply_http_endpoint = ("https://www.googleapis.com/youtube/v3/comments?"
                         "part=snippet&textFormat=plainText&maxResults=100&"
                         "parentId={}&key={}".format(comment['comment_id'],key))
            response = requests.get(reply_http_endpoint)
            json_doc = json.loads(response.text)
            comments.extend([parse_comment_metadata(comment) for comment in json_doc['items']])
            
            
    return comments

In [70]:
comet = get_comments(video_id, key)

In [71]:
len(comet)

679

In [72]:
comet[:10]

[OrderedDict([('commenter_channel_url',
               'http://www.youtube.com/channel/UCV1FkeifGocTQp3UUTj4B3A'),
              ('commenter_channel_display_name', 'Oopss7'),
              ('comment_id', 'UgwAvyGsdNUM7aUTJoF4AaABAg'),
              ('comment_like_count', 0),
              ('comment_publish_date',
               datetime.datetime(2018, 6, 2, 18, 45, 21)),
              ('text',
               'The reference to the "you, senator, are no JFK right at the end"'),
              ('video_id', 'SH9v7zauRH4'),
              ('commenter_rating', 'none'),
              ('comment_parent_id', None),
              ('collection_date',
               datetime.datetime(2018, 6, 2, 14, 48, 31, 382114))]),
 OrderedDict([('commenter_channel_url',
               'http://www.youtube.com/channel/UCZa6r_r7u-lKvCtZars4fOg'),
              ('commenter_channel_display_name', 'Isasiah Bradley'),
              ('comment_id', 'UgyDB5133Oz330bygjB4AaABAg'),
              ('comment_like_count', 0),
 